# Simulating resting-state network dynamics with The Virtual Brain
## Introduction

Fluctuations in brain activity in non-task conditions are now a well-established phenomena in the literature. These fluctuations are not random but shown to exhibit spatial patterns, referred to as resting state networks. Despite being readily identifiable during rest, these networks are related to specific functions and on the other hand abnormalities in such RSNs have been associated with pathology.

In the following, we will demonstrate some starting points for modeling resting state networks in TVB, using the default data set.

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.decomposition import FastICA
from tvb.basic.logger.builder import get_logger
from tvb.datatypes.connectivity import Connectivity
from tvb.simulator import coupling, integrators, models, monitors, noise

import utils

LOG = get_logger("demo")

## Setting up the simulation

In the following, we'll use a basic region level simulation, with the generic oscillator set in an excitable regime, linear coupling with low strength, a stochastic integrator with low noise and a temporal average monitor at 200 Hz.

These settings are a good starting point for modeling resting state patterns because no particular factor dominates the dynamics and a balance between the structural connectivity, moderate intrinsic excitability and noise comes into play.

### The TVB Simulator Object
- model: Neural Mass Model defining local neural dynamics
- connectivity: Structural Connectivity defining the network
- coupling: Coupling function defining how coupling is done
- integrator: Integrator defining how integration is done
- monitors: Temporal Average Monitor to log the data
  

In [ ]:
from tvb.simulator.simulator import Simulator


def run_sim(conn, cs, D, cv=3.0, dt=0.5, simlen=1e3):
    sim = Simulator(
        model=models.Generic2dOscillator(a=np.array([0.0])),
        connectivity=conn,
        coupling=coupling.Linear(a=np.array([cs])),
        integrator=integrators.HeunStochastic(
            dt=dt, noise=noise.Additive(nsig=np.array([D]))
        ),
        monitors=(monitors.TemporalAverage(period=5.0),),  # 200 Hz
    )
    sim.configure()
    ((t, y),) = sim.run(simulation_length=simlen)
    return t, y[:, 0, :, 0]


conn = Connectivity.from_file()

One of the common features of simulations is an initial transient, so we'll perform a one minute simulation, and as soon as the time series have been generated, we check that the transient has decayed:

In [ ]:
tic = time.time()
t, y = run_sim(conn, cs=6e-2, D=5e-4, simlen=1 * 60e3)
"simulation required %0.3f seconds." % (time.time() - tic,)

# Functional Connectivity

Next, to quickly assess the presence of a network structure in the time series, we window the time series into 1 second non overlapping windows, obtain per-window correlation matrices

In [ ]:
def calculate_fc(t, y):
    cs = []
    for i in range(int(t[-1] / 1e3)):
        cs.append(np.corrcoef(y[(t > (i * 1e3)) * (t < (1e3 * (i + 1)))].T))
    cs = np.array(cs)
    return cs


def ttest(cs):
    cs_z = np.arctanh(cs)
    for i in range(cs.shape[1]):
        cs_z[:, i, i] = 0.0
    _, p = stats.ttest_1samp(cs, 0.0)
    return p

The strength of correlation can be assessed statistically by Fisher Z transforming the coefficients and applying a t-test,

In [ ]:
cs = calculate_fc(t, y)
p = ttest(cs)

Which we then visualize the structural connectivity (left) and functional connectivity (right) as an adjacency matrices applying a threshold on significance:

In [ ]:
fig, axs = plt.subplots(ncols=4, figsize=(12, 4))

# First subplot: Connectivity weights
im0 = axs[0].imshow(conn.weights, cmap="inferno")
axs[0].set_title("Connectivity Weights")
fig.colorbar(im0, ax=axs[0], shrink=0.5)

# Second subplot: Correlation Coefficient Matrix
im1 = axs[1].imshow(np.corrcoef(y[1000:], rowvar=False), cmap="inferno")
axs[1].set_title("Simulated FC")
fig.colorbar(im1, ax=axs[1], shrink=0.5)

# Third subplot: P-values
im2 = axs[2].imshow(p * (p < 0.05), cmap="inferno")
axs[2].set_title("P-values")
fig.colorbar(im2, ax=axs[2], shrink=0.5)

# Overlay: Log10 of P-values (p < 0.05)
im3 = axs[3].imshow(np.log10(p) * (p < 0.05), cmap="inferno")
axs[3].set_title("Log10 P-values (p < 0.05)")
fig.colorbar(im3, ax=axs[3], shrink=0.5)


plt.show()

We can see there are significant deviations in the FC from the SC which are especially prominent in the interhemispheric FC, where interactions are found despite limited interhemispheric SC.

We can then ask what degree of similarity there is between the average functional connectivity and the structural connectivity, and how it varies over time:

In [ ]:
plt.figure()
plt.plot(
    np.r_[1 : len(cs) + 1],
    [np.corrcoef(cs_i.ravel(), conn.weights.ravel())[0, 1] for cs_i in cs],
)
plt.ylim([0, 0.5])
plt.ylabel("FC-SC correlation")
plt.xlabel("Time (s)")
plt.show()

# Seed-region correlation maps

A common visualization of FC specific to a given is to pull out its row of the FC matrix and plot a colormap on the anatomy. We can do this will the simulated functional connectivity to identify which regions are highly coordinated with the seed region.

In [ ]:
from tvb.datatypes import cortex

ctx = cortex.Cortex.from_file()
ctx.region_mapping_data.connectivity = conn
rm = ctx.region_mapping


def plot_roi_corr_map(reg_name):
    roi = list(conn.ordered_labels).index(reg_name)
    cs_m = cs[2:].mean(axis=0)
    utils.multiview(
        cs_m[roi][rm], cortex=ctx, shaded=False, suptitle=reg_name, figsize=(10, 5)
    )

As a few examples of such maps, seeding in the left motor cortex, right ventrolateral prefront cortex, and right superior parietal cortex:

In [ ]:
for reg in "lM1 rPFCVL rPCS".split():
    plot_roi_corr_map(reg)

Seed-region maps are useful when one has a prior about which regions are implicated for a given network.

# ICA

Another common exploratory tool in resting state data analysis, where the implicated regions or networks are not known a priori, is independent component analysis, which extracts components with unique or independent statistical properties.

For exapmle, we can perform an ICA keeping 5 components the above simulated data,

In [ ]:
ica = FastICA(n_components=5, max_iter=250)
ica.fit(y[t > 2e3])

And then visualize the different components

In [ ]:
for i, comp in enumerate(ica.components_[:4]):
    utils.multiview(
        comp[rm], cortex=ctx, shaded=False, suptitle="ICA %d" % (i,), figsize=(10, 5)
    )

These components are not selected 'by hand', but represent independent subnetworks during the simulated resting state activity.

Finally, we point out, that commonly ICA analyses of fMRI are done at a group level to identify spatial patterns which are reproducible across subjects, whereas in this application to this simulation, spatial components may reflect as much non-robust, spontaneous fluctuations of the network passing through various state as the dominant rest state networks identified in human rest state.

# Further exploration

These results are starting point, from which you can base you simulations, going in directions such as

- Perform a parameter sweep to identify regions of improved FC-SC correlation
- Introduce a subject specific structural connectivity and compare with subject specific functional connectivity
- Threshold the calculated functional connectivities and apply graph theoretic measures

We hope this has been a useful tutorial and welcome any comments or questions on our mailing list (https://groups.google.com/forum/#!forum/tvb-users).

# References

- [Damoiseaux, J. S., Rombouts, S. A. R. B., Barkhof, F., Scheltens, P., Stam, C. J., Smith, S. M., & Beckmann, C. F. (2006). Consistent resting-state networks across healthy subjects. Proceedings of the national academy of sciences, 103(37), 13848-13853.](http://www.pnas.org/content/103/37/13848.full)
- [Mantini, D., Perrucci, M. G., Del Gratta, C., Romani, G. L., & Corbetta, M. (2007). Electrophysiological signatures of resting state networks in the human brain. Proceedings of the National Academy of Sciences, 104(32), 13170-13175](https://www.pnas.org/content/104/32/13170.full)
- [Ghosh, A., Rho, Y., McIntosh, A. R., Kötter, R., & Jirsa, V. K. (2008). Noise during rest enables the exploration of the brain's dynamic repertoire. PLoS Comput Biol, 4(10), e1000196.](http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1000196)
- [Honey, C. J., Sporns, O., Cammoun, L., Gigandet, X., Thiran, J. P., Meuli, R., & Hagmann, P. (2009). Predicting human resting-state functional connectivity from structural connectivity. Proceedings of the National Academy of Sciences, 106(6), 2035-2040.](https://www.pnas.org/content/106/6/2035.full)
- [Deco, G., & Jirsa, V. K. (2012). Ongoing cortical activity at rest: criticality, multistability, and ghost attractors. The Journal of Neuroscience, 32(10), 3366-3375.](www.jneurosci.org/content/32/10/3366.short)
- [Hansen, E. C., Battaglia, D., Spiegler, A., Deco, G., & Jirsa, V. K. (2015). Functional connectivity dynamics: modeling the switching behavior of the resting state. Neuroimage, 105, 525-535.](http://www.sciencedirect.com/science/article/pii/S1053811914009033)